# Code to download part of EM-EARTH data for a given variable(s), year(s), or region(s)


## For using the code, linux system is needed and wget should be available.


## How to Run

### 1 use the jupyter notebook

The arguments to `run_download` should be refined, otherwise default values generated by `initialize_variables` will be used

### 2 convert to python and use command line on linux system

```
jupyter nbconvert download_EM_EARTH.ipynb --to python # convert to python
```

Example 1 - download all the reagions of the world for February of 1970 for probabilistic daily for two ensmebl memebers out of the 25
```
python download_EM_EARTH.py --years ['1970'] --months ['02'] --ensembles ['001'] --groups ['probabilistic_daily'] 
```
Example 2 - download all the reagions of Africa for February of 1970 for probabilistic daily for all the ensemble members =
```
python download_EM_EARTH.py --years ['1970'] --months ['02'] --groups ['probabilistic_daily'] --regions ['Africa']
```

## Note: the virtual env should be created and dependecies should be installed


In [8]:
##
import os
import hashlib
import glob
import pandas as pd
import re
#import gdown
import click
import psutil



def initialize_variables ():
    
    server = 'https://g-772fa5.cd4fe.0ec8.data.globus.org/6/published/publication_542/submitted_data/'
    groups = ['deterministic_hourly','deterministic_raw_daily','probabilistic_daily']
    variables = ['prcp','tdew','tmean','trange']
    regions = ['Asia','Europe','Africa','NorthAmerica','Oceania','SouthAmerica']
    years = ['1950','1951','1952','1953','1954',\
             '1955','1956','1957','1958','1959',\
             '1960','1961','1962','1963','1964',\
             '1965','1966','1967','1968','1969',\
             '1970','1971','1972','1973','1974',\
             '1975','1976','1977','1978','1979',\
             '1980','1981','1982','1983','1984',\
             '1985','1986','1987','1988','1989',\
             '1990','1991','1992','1993','1994',\
             '1995','1996','1997','1998','1999',\
             '2000','2001','2002','2003','2004',\
             '2005','2006','2007','2008','2009',\
             '2010','2011','2012','2013','2014',\
             '2015','2016','2017','2018','2019']
    months = ['01','02','03','04','05','06',\
              '07','08','09','10','11','12']
    ensembles = ['001','002','003','004','005',\
                 '006','007','008','009','010',\
                 '011','012','013','014','015',\
                 '016','017','018','019','020',\
                 '021','022','023','024','025']
    root_save = '../EM_Earth_v1/'
    
    return server, groups, variables, regions, years, months, ensembles, root_save



def download_EM_ERATH (server = 'https://g-772fa5.cd4fe.0ec8.data.globus.org/6/published/publication_542/submitted_data/',\
                       groups = ['deterministic_hourly','deterministic_raw_daily',\
                                 'probabilistic_daily'],\
                       variables = ['prcp','tdew','tmean','trange'],\
                       regions = ['Asia','Europe','Africa',\
                                  'NorthAmerica','Oceania','SouthAmerica'],\
                       years = ['1950','1951','1952','1953','1954',\
                                '1955','1956','1957','1958','1959',\
                                '1960','1961','1962','1963','1964',\
                                '1965','1966','1967','1968','1969',\
                                '1970','1971','1972','1973','1974',\
                                '1975','1976','1977','1978','1979',\
                                '1980','1981','1982','1983','1984',\
                                '1985','1986','1987','1988','1989',\
                                '1990','1991','1992','1993','1994',\
                                '1995','1996','1997','1998','1999',\
                                '2000','2001','2002','2003','2004',\
                                '2005','2006','2007','2008','2009',\
                                '2010','2011','2012','2013','2014',\
                                '2015','2016','2017','2018','2019'],\
                       months = ['01','02','03','04','05','06',\
                                 '07','08','09','10','11','12'],\
                       ensembles = ['001','002','003','004','005',\
                                    '006','007','008','009','010',\
                                    '011','012','013','014','015',\
                                    '016','017','018','019','020',\
                                    '021','022','023','024','025'],\
                       root_save = '../EM_Earth_v1/'):
    
    # string to list
    groups     =  string_to_list(groups)
    variables  =  string_to_list(variables)
    regions    =  string_to_list(regions)
    years      =  string_to_list(years)
    months     =  string_to_list(months)
    ensembles  =  string_to_list(ensembles)
    
    # get the csv
    url = "https://drive.google.com/uc?id=1HWNNCBpalAymIouburwm7flDhcjqWcqN"
    output = "../hash/frdr-checksums-0547.csv"
    if not os.path.isdir('../hash/'):
        os.makedirs('../hash/')
    gdown.download(url, output, quiet=False)
    df = pd.read_csv(output)
    
    # 
    for group in groups:
        for year in years:
            for month in months:
                for variable in variables:
                    
                    # if the case is deterministic raw daily
                    if group == 'deterministic_raw_daily':
                        #
                        terms = [group, year+month, variable] # terms are the terms 
                        df_slice = df.copy()
                        for term in terms: # loop over terms to get the target file names
                            #
                            df_slice = df_slice[df_slice['File_Name'].str.contains(term)]
                        
                        for index, row in df_slice.iterrows():
                            
                            file_name, path_name, link_name =                             prepare_file_name_path_name(row['File_Name'],root_save,server)
                            
                            downlaod (file_name, path_name, link_name, row['sha256'].strip())
                            
                    
                    
                    for region in regions:
                        if group == 'deterministic_hourly' and variable != 'trange':
                            #
                            terms = [group, year+month, variable, region] # terms are the terms 
                            df_slice = df.copy()
                            
                            for term in terms: # loop over terms to get the target file names
                                #
                                df_slice = df_slice[df_slice['File_Name'].str.contains(term)]
                        
                            for index, row in df_slice.iterrows():

                                file_name, path_name, link_name =                                 prepare_file_name_path_name(row['File_Name'],root_save,server)

                                downlaod (file_name, path_name, link_name, row['sha256'].strip())
                                
                        if group == 'probabilistic_daily':

                            for ensemble in ensembles:

                                #
                                terms = [group, year+month, variable, region, '_'+ensemble] # terms are the terms 
                                df_slice = df.copy()
                                
                                for term in terms: # loop over terms to get the target file names
                                    #
                                    df_slice = df_slice[df_slice['File_Name'].str.contains(term)]
                        
                                for index, row in df_slice.iterrows():

                                    file_name, path_name, link_name =                                     prepare_file_name_path_name(row['File_Name'],root_save,server)

                                    downlaod (file_name, path_name, link_name, row['sha256'].strip())

def get_hash (file_name):
    sha256_hash = hashlib.sha256()
    with open(file_name,"rb") as f:
        # Read and update hash string value in blocks of 4K
        for byte_block in iter(lambda: f.read(4096),b""):
            sha256_hash.update(byte_block)
        return sha256_hash.hexdigest().strip()
    
def prepare_file_name_path_name(full_name,
                                root_save,
                                server):
    file_path = re.sub('/globusdata/6/published/publication_542/submitted_data/EM_Earth_v1/',                       '',                       full_name)
    file_name = file_path.split("/")[-1].strip()
    path_name = re.sub(file_name,'',file_path).strip()
    link_name = server+'EM_Earth_v1/'+path_name+file_name
    path_name = root_save+path_name
    
    return file_name, path_name, link_name

def downlaod (file_name, path_name, link_name, hash_value_remote):
    
    #
    downloaded = False
    try_number = 1
            
    
    # directory
    if not os.path.isdir(path_name):
        os.makedirs(path_name)
    
    # check if the file exists on local directory
    if os.path.isfile(path_name+file_name):
        
        # check the hash value
        hash_value_local = get_hash (path_name+file_name)
        
        if hash_value_local == hash_value_remote:
            
            #
            downloaded = True
            
    while (not downloaded) and (try_number < 1000):

        os.system('wget '+link_name+' -O '+path_name+file_name)

        hash_value_local = get_hash (path_name+file_name)
        
        print(hash_value_local)
        print(hash_value_remote)

        if hash_value_local == hash_value_remote:
            downloaded = True

        try_number = try_number + 1




def string_to_list (string_in):
    
    if isinstance(string_in, str):
        string_in = string_in.replace("[", "")
        string_in = string_in.replace("]", "")
        string_in = string_in.replace(",", " ")
        string_in = string_in.replace("'", " ")
        list_out  = string_in.split()
        
        return list_out
    else:
        return string_in
    


@click.command()
@click.option('--server', 'server',    default  =  'https://g-772fa5.cd4fe.0ec8.data.globus.org/6/published/publication_542/submitted_data/')
@click.option('--groups', 'groups',    default  =  ['deterministic_hourly',\
                                                    'deterministic_raw_daily',\
                                                    'probabilistic_daily'])
@click.option('--variables', 'variables', default  =   ['prcp','tdew','tmean','trange'])
@click.option('--regions', 'regions', default  =   ['Asia','Europe','Africa',\
                                                    'NorthAmerica','Oceania','SouthAmerica'])
@click.option('--years', 'years', default  =   ['1950','1951','1952','1953','1954',\
                                                '1955','1956','1957','1958','1959',\
                                                '1960','1961','1962','1963','1964',\
                                                '1965','1966','1967','1968','1969',\
                                                '1970','1971','1972','1973','1974',\
                                                '1975','1976','1977','1978','1979',\
                                                '1980','1981','1982','1983','1984',\
                                                '1985','1986','1987','1988','1989',\
                                                '1990','1991','1992','1993','1994',\
                                                '1995','1996','1997','1998','1999',\
                                                '2000','2001','2002','2003','2004',\
                                                '2005','2006','2007','2008','2009',\
                                                '2010','2011','2012','2013','2014',\
                                                '2015','2016','2017','2018','2019'])
@click.option('--months', 'months', default  =    ['01','02','03','04','05','06',\
                                                   '07','08','09','10','11','12'])
@click.option('--ensembles', 'ensembles', default  =    ['001','002','003','004','005',\
                                                        '006','007','008','009','010',\
                                                        '011','012','013','014','015',\
                                                        '016','017','018','019','020',\
                                                        '021','022','023','024','025'])
@click.option('--root_save', 'root_save', default  =    '../EM_Earth_v1/')
def download_EM_ERATH_command( server,\
                               groups,\
                               variables,\
                               regions,\
                               years,\
                               months,\
                               ensembles,\
                               root_save):
    
    download_EM_ERATH(server      =  server,\
                      groups      =  groups,\
                      variables   =  variables,\
                      regions     =  regions,\
                      years       =  years,\
                      months      =  months,\
                      ensembles   =  ensembles,\
                      root_save   =  root_save)

    


# def runninginJupyterNotebook():
#     for i in psutil.Process().parent().cmdline():
#         if i.endswith("bin/jupyter-notebook"):
#             return True
#     else:
#         return False


def run_download (server     ,\
                  groups     ,\
                  variables  ,\
                  regions    ,\
                  years      ,\
                  months     ,\
                  ensembles  ,\
                  root_save  ):
    
#     # check the 
#     notebook = runninginJupyterNotebook()
    
#     if notebook:
#         download_EM_ERATH(server      =  server,\
#                           groups      =  groups,\
#                           variables   =  variables,\
#                           regions     =  regions,\
#                           years       =  years,\
#                           months      =  months,\
#                           ensembles   =  ensembles,\
#                           root_save   =  root_save)
    # else:
    #if __name__ == '__main__':
    #    # initilize the varibales
    download_EM_ERATH_command()



In [9]:
#################
#### RUN DOWNLAOD
#################

# initlize paramaters
server, groups, variables, regions, years, months, ensembles, root_save = initialize_variables()

# assign the varibales or regions or ensmeblse to downs,
# if commneted or not specify the default will be used
years = ['1960']
months = ['01']
#groups = ['probabilistic_daily']
#ensembles = ['001']


# simulate
run_download (server     ,\
              groups     ,\
              variables  ,\
              regions    ,\
              years      ,\
              months     ,\
              ensembles  ,\
              root_save  )


Usage: ipykernel_launcher.py [OPTIONS]
Try 'ipykernel_launcher.py --help' for help.

Error: No such option: -f


AssertionError: 